In [1]:
import random
import numpy as np
import crocoddyl
import keras
from keras.models import load_model
from keras import regularizers
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
crocoddyl.switchToNumpyMatrix();

Using TensorFlow backend.
/home/a/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/a/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/a/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/a/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py

In [2]:
def iteration_analysis(MAXITER:int =1000):
    """
    Script to store the number of iterations crocoddyl takes to generate a trajectory.
    Here, we generate 10K trajectories and store the number of iterations taken by  crocoddyl 
    for each trajectory
    
    """
    iterations = []
    not_feasible = 0
    
    NUM_TRAJ = 10000


    for _ in range(NUM_TRAJ):
        initial_config = [random.uniform(-2.1, 2.),
                                    random.uniform(-2.1, 2.), 
                                    random.uniform(0, 1)]

        model = crocoddyl.ActionModelUnicycle()

        model.costWeights = np.matrix([1, 0.3]).T

        problem = crocoddyl.ShootingProblem(np.matrix(initial_config).T, [ model ] * 20, model)

        ddp = crocoddyl.SolverDDP(problem)

        done = ddp.solve([], [], MAXITER)
        if not ddp.isFeasible:
            not_feasible += 1

        else:
            iterations.append(ddp.iter)

    max_value = max(iterations)
    min_value = min(iterations)
    avg_value = sum(iterations)//len(iterations)
    print(f"Total trajectories: {NUM_TRAJ}, not feasible trajectories: {not_feasible}\n\n")
    print(f'Maximum iterations allowed : {MAXITER}\n')
    print(f"\nMinimum iterations taken -> {min_value},\
          Maximum iterations taken -> {max_value},\
          Average -> {avg_value} \n")
    print(".............................\n")
    
    return iterations

Group1 = iteration_analysis(MAXITER = 1000)
Group2 = iteration_analysis(MAXITER = 100)
Group3 = iteration_analysis(MAXITER = 500)

Total trajectories: 10000, not feasible trajectories: 0


Maximum iterations allowed : 1000


Minimum iterations taken -> 2,          Maximum iterations taken -> 409,          Average -> 25 

.............................

Total trajectories: 10000, not feasible trajectories: 0


Maximum iterations allowed : 100


Minimum iterations taken -> 2,          Maximum iterations taken -> 100,          Average -> 25 

.............................

Total trajectories: 10000, not feasible trajectories: 0


Maximum iterations allowed : 500


Minimum iterations taken -> 2,          Maximum iterations taken -> 470,          Average -> 26 

.............................



In [3]:
def generate_data(ntrajectories: int = 10000,
                  maxiter: int = 100,
                  knots: int = 20,
                  state_weight: float = 1., # can use random.uniform(1, 3)
                  control_weight: float = 0.3, # can use random.uniform(0,1)
                 ):
    """
    @ Arguments:
        ntrajectories : number of trajectories to generate
        maxiter       : number of iterations crocoddyl is supposed to make for each trajectory
        knots         : Knots, e.g T = 20, 30
        
        
    @ Returns:
        initial_states         : matrix of starting points, shape (ntrajectories, 3)....x_data
        optimal trajectories   : matrix of trajectories, shape (ntrajectores, knots).....y_data
                                 Each row of this matrix is:
                                     x1, y1, theta1, v1, w1, x2, y2, theta2, v2, w2, .....so on
        
    """
    starting_configurations = []
    optimal_trajectories = []
    
    for _ in range(ntrajectories):
            initial_config = [random.uniform(-2.1, 2.),
                                        random.uniform(-2.1, 2.), 
                                        random.uniform(0, 1)]
            
            model = crocoddyl.ActionModelUnicycle()
            model.costWeights = np.matrix([state_weight, control_weight]).T
            problem = crocoddyl.ShootingProblem(np.matrix(initial_config).T, [ model ] * knots, model)
            ddp = crocoddyl.SolverDDP(problem)
            ddp.solve([], [], maxiter)
            
            if ddp.isFeasible:
                state = np.array(ddp.xs)
                control = np.array(ddp.us)

                optimal_trajectory = np.hstack((state[1:,:], control))
                optimal_trajectory = np.ravel(optimal_trajectory)
                optimal_trajectories.append(optimal_trajectory)
                starting_configurations.append(initial_config)
                
    optimal_trajectories = np.array(optimal_trajectories).clip(min=1e-10)
    starting_configurations = np.array(starting_configurations).clip(min=1e-10)         
    return starting_configurations, optimal_trajectories
    

In [ ]:

def train_net(n_hidden: int = 4, save_model: bool = False):
    """
    A generic keras 4 hidden layered neural net with RMSprop as optimizer
    Each Layer has 256 neurons.
    
    """
    x_data, y_data = generate_data()
    
    
    model = keras.models.Sequential()
    
    model.add(Dense(256, input_dim=(x_data.shape[1])))
    model.add(Activation('relu'))
    for _ in range(n_hidden):
        model.add(Dense(256,
                        activation = "tanh",
                        kernel_initializer='random_uniform',
                        kernel_regularizer=regularizers.l2(0.01),
                        activity_regularizer=regularizers.l1(0.01)))            
        model.add(Dropout(0.25))

    model.add(Dense(y_data.shape[1], 
                    activation = 'linear'))        


    rms = optimizers.RMSprop(learning_rate=0.001, rho=0.9)
    model.compile(loss='mean_squared_error',
                  optimizer=rms,
                  metrics=['mean_squared_error', "mean_absolute_error"])
    
    model.fit(x_data, 
                  y_data,
                  epochs = 200,
                  batch_size= 16,
                  verbose = 0
                  )
        
    #x_test, y_test = generate_data(1000) 
    #score = model.evaluate(x_test, y_test, batch_size = 16, use_multiprocessing=True)
    
    if save_model:
        model.save('basic_model.h5')
        
    else: return model    
            
train_net(save_model=True)

In [9]:

def warmstart(n_trajectories: int = 10000, load_net: bool = False):
    if load_net:
        neuralNet = load_model('basic_model.h5')
    else:
        neuralNet = train_net()
        
    print(f" With warmstart, without warmstart ")
    
    for _ in range(n_trajectories):
            
            initial_config = [random.uniform(-2.1, 2.),
                                        random.uniform(-2.1, 2.), 
                                        random.uniform(0, 1)]

            model = crocoddyl.ActionModelUnicycle()
            model2 = crocoddyl.ActionModelUnicycle()
            
            model.costWeights = np.matrix([1., .3]).T
            model2.costWeights = np.matrix([1., .3]).T
    
            
            problem = crocoddyl.ShootingProblem(np.matrix(initial_config).T, [ model ] * 20, model)
            problem2 = crocoddyl.ShootingProblem(np.matrix(initial_config).T, [ model2 ] * 20, model2)

            
            ddp = crocoddyl.SolverDDP(problem)
            ddp2 = crocoddyl.SolverDDP(problem2)
            
            
            x_data = np.matrix(initial_config).reshape(1, 3)

            prediction = neuralNet.predict(x_data).reshape(20, 5)

            xs = np.matrix(prediction[:, 0:3])

            us = np.matrix(prediction[:, 3:5])
            

            ddp_xs = []
            ddp_xs.append(np.matrix(initial_config).reshape(1, 3).T)
            for _ in  range(xs.shape[0]):
                ddp_xs.append((np.matrix(xs[_]).T))

            ddp_us = []
            for _ in  range(us.shape[0]):
                ddp_us.append(np.matrix(us[_]).T)

            ddp.solve(ddp_xs,ddp_us, 100)
            ddp2.solve()
            print("  ", ddp.iter, "  ", ddp2.iter)
                
    

In [10]:
warmstart(n_trajectories=5000, load_net=True)

 With warmstart, without warmstart 
   20    19
   8    8
   46    42
   55    48
   46    22
   8    8
   9    9
   4    3
   15    14
   17    15
   24    28
   6    6
   27    26
   16    17
   80    98
   17    17
   54    100
   12    14
   23    23
   12    15
   31    28
   8    9
   21    19
   66    77
   8    8
   54    49
   6    5
   7    7
   7    6
   19    23
   21    21
   9    8
   55    67
   7    6
   23    20
   5    5
   6    5
   36    26
   62    51
   86    80
   12    12
   25    26
   19    18
   28    32
   52    69
   97    100
   14    13
   16    20
   40    52
   28    33
   22    23
   72    37
   16    13
   12    10
   33    32
   4    5
   10    11
   7    8
   13    16
   23    25
   64    97
   16    15
   53    62
   60    63
   41    48
   29    25
   8    8
   8    10
   12    11
   17    17
   34    34
   74    80
   37    37
   58    57
   35    43
   13    13
   20    19
   6    6
   100    78
   35    32
   17    15
   16    15
   17    18
  

   55    48
   7    7
   22    23
   76    84
   19    21
   23    20
   11    11
   52    41
   86    42
   11    12
   20    27
   11    11
   65    68
   49    60
   22    22
   80    71
   18    19
   15    13
   38    39
   100    54
   6    6
   7    7
   69    61
   26    27
   13    13
   13    11
   5    4
   25    27
   100    46
   9    9
   23    25
   25    26
   32    30
   39    32
   20    18
   6    6
   17    17
   5    5
   30    28
   8    7
   16    20
   8    8
   42    51
   13    13
   54    22
   25    24
   22    25
   34    39
   5    6
   51    42
   5    6
   4    4
   12    14
   66    43
   8    10
   7    7
   12    12
   33    30
   29    35
   37    29
   100    44
   12    11
   6    5
   100    72
   6    6
   12    10
   6    7
   17    25
   58    43
   63    57
   22    23
   95    74
   12    11
   5    6
   34    28
   19    22
   22    21
   56    65
   5    6
   66    55
   8    7
   13    13
   21    24
   20    24
   8    8
   29    26
   92

   38    32
   20    19
   11    11
   20    19
   10    12
   40    47
   17    21
   8    8
   8    8
   45    38
   11    9
   17    19
   32    25
   5    6
   46    25
   45    39
   25    28
   20    24
   9    10
   13    14
   8    8
   14    14
   32    36
   61    51
   13    11
   13    17
   45    42
   11    10
   29    33
   6    6
   54    48
   17    17
   10    11
   7    6
   7    6
   80    68
   49    53
   9    9
   22    21
   50    42
   48    47
   25    22
   10    12
   6    6
   8    9
   38    41
   12    13
   31    29
   8    8
   57    41
   66    77
   31    31
   15    15
   29    27
   20    19
   14    14
   36    40
   17    20
   35    44
   5    5
   28    32
   23    27
   44    41
   23    23
   6    6
   47    44
   6    6
   3    3
   7    8
   49    35
   42    44
   47    50
   9    9
   59    41
   23    22
   30    27
   13    13
   8    7
   12    13
   21    21
   25    20
   37    46
   12    16
   100    74
   36    42
   39    34
   37

   23    24
   26    29
   10    11
   18    21
   17    17
   22    19
   44    41
   45    54
   64    31
   42    34
   19    21
   15    14
   86    50
   89    24
   50    40
   30    29
   21    20
   47    28
   24    26
   32    29
   10    13
   16    15
   25    28
   32    26
   88    27
   12    12
   20    16
   32    40
   29    35
   6    6
   28    26
   9    10
   9    9
   75    44
   7    7
   17    16
   76    53
   24    21
   4    3
   24    30
   22    21
   55    65
   11    12
   26    24
   43    34
   34    32
   20    23
   75    31
   33    34
   10    10
   92    72
   21    20
   6    6
   20    17
   5    5
   6    6
   46    51
   54    32
   8    8
   22    21
   14    14
   8    9
   9    10
   33    24
   66    51
   10    9
   7    7
   13    12
   22    23
   5    4
   9    9
   15    19
   34    24
   59    55
   46    38
   21    20
   31    34
   86    79
   8    8
   12    14
   20    22
   14    13
   17    16
   37    29
   21    20
   23    

   28    33
   18    17
   5    4
   57    49
   51    59
   85    100
   9    11
   100    100
   30    26
   27    26
   39    38
   19    19
   15    13
   32    34
   60    21
   36    35
   7    9
   51    50
   75    72
   4    4
   8    8
   21    21
   22    20
   31    36
   18    17
   37    30
   50    47
   7    6
   17    19
   49    45
   16    22
   12    13
   13    13
   5    5
   10    9
   32    38
   37    35
   13    11
   13    13
   8    7
   7    5
   30    28
   4    3
   7    7
   63    63
   29    28
   5    5
   30    27
   71    60
   14    14
   64    69
   16    25
   21    23
   38    31
   14    12
   26    32
   40    39
   16    15
   34    36
   21    23
   5    5
   42    40
   30    28
   49    77
   41    38
   28    33
   19    18
   35    32
   3    3
   31    28
   43    49
   29    27
   27    30
   11    11
   39    38
   23    22
   30    22
   11    10
   5    4
   4    4
   13    12
   20    20
   7    7
   87    87
   20    20
   7    7
 

   13    12
   23    24
   9    10
   20    20
   91    65
   23    23
   37    32
   100    100
   13    14
   85    20
   21    22
   12    16
   22    25
   21    20
   48    45
   46    24
   36    36
   7    6
   11    12
   22    25
   9    8
   16    21
   61    65
   8    7
   13    13
   36    30
   7    6
   24    22
   25    26
   15    13
   11    10
   17    16
   24    28
   16    15
   18    18
   55    41
   41    48
   15    18
   40    47
   13    12
   39    24
   5    5
   32    31
   6    6
   19    22
   60    45
   29    33
   18    17
   43    52
   13    12
   31    27
   23    24
   10    10
   26    24
   72    84
   7    7
   12    12
   76    50
   100    65
   30    27
   28    25
   49    43
   5    5
   19    22
   8    9
   7    6
   97    59
   46    42
   41    22
   25    24
   48    42
   18    15
   74    64
   32    30
   11    12
   100    21
   34    32
   9    8
   12    12
   49    41
   11    13
   33    28
   69    67
   5    4
   14    13
 

   11    10
   5    6
   13    13
   26    22
   20    19
   21    24
   22    19
   15    17
   18    17
   16    14
   5    6
   91    38
   100    39
   28    28
   8    8
   14    15
   27    27
   54    57
   23    24
   6    6
   30    36
   19    18
   33    29
   21    19
   51    69
   9    9
   20    23
   7    6
   46    22
   63    54
   9    9
   46    56
   81    71
   43    20
   62    26
   33    34
   8    9
   14    16
   25    23
   69    76
   35    46
   21    18
   3    3
   28    27
   15    17
   5    4
   7    7
   18    20
   8    7
   19    18
   10    9
   31    34
   5    6
   38    46
   15    14
   40    35
   22    25
   17    18
   8    7
   10    8
   7    8
   13    13
   3    3
   25    22
   19    18
   5    6
   32    28
   3    3
   10    9
   35    45
   8    7
   13    12
   26    25
   21    22
   26    27
   69    42
   55    38
   25    24
   35    41
   62    55
   24    23
   43    35
   26    26
   22    27
   24    24
   19    21
   14   